Libraries  loading

In [2]:
# !conda install -c conda-forge folium=0.5.0 --yes
!pip install folium

     |████████████████████████████████| 92kB 8.4MB/s eta 0:00:011


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# The Battle of Neighborhoods

## Introduction

Lin will be assigned to oversea as a contactor of his company for three months. He has to decide which city he may locate from Toronto, Paris and New York. Because it'll be a long term trip to North America and he likes Asian food very much, so he wants to know which city is better for his travel. That means which city has a high-quantity and high-quality Asian cuisine restaurant. He managed to compare the neighborhoods of the two cities and determine how similar or dissimilar they are before his travel, And then he can get more helpful information for him making decisions.

## Data section

Foursquare API is used to fetch and explore neighborhood data of those three cities. The information he wants to focus on is restaurants. He will choose the city's coordinate as the start point. Analyzing the neighborhood data and prioritize the best restaurant location in those three cities based on the number of restaurants, average distance, and average rate. Lastly, he can decide which city is the better place for him to get start.

## Methodology

There are two main methods are used to get data from the Foursquare API.

The first one is Search for a specific venue category. > `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**. He will get venues from the API base on given LATITUDE, LONGITUDE, and RADIUS. In this way, you can get all the restaurants are surrounding the city coordinate.

The second one is to Explore a Given Venue. > `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**. He will get the rate of each restaurant base on their unique id.

In [89]:
CLIENT_ID = 'LC5NMEW0G3EXBTMDFFC122PMILOAAJ0C4Q2RQJRWEK4IWA1T' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

My credentails:
CLIENT_ID: LC5NMEW0G3EXBTMDFFC122PMILOAAJ0C4Q2RQJRWEK4IWA1T
CLIENT_SECRET: 


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>foursquare_agent</em>, as shown below.

In [5]:
cities = ['Toronto, ON, Canada', 'Paris, France', 'New York City, NY']

In [31]:
search_query = 'Asian'
radius = 5000
LIMIT = 30
print(search_query + ' .... OK!')

Asian .... OK!


## Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

In [80]:
geolocator = Nominatim(user_agent="foursquare_agent")

for city in cities:
    location = geolocator.geocode(city)
    latitude = location.latitude
    longitude = location.longitude
    city_coor = [latitude, longitude] 
    print('The geograpical coordinate of {} are {}, {}.'.format(city, latitude, longitude))
    
    # Search for a specific venue category
    # Define the corresponding URL
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    # print(url)
    
    # Send the GET Request and examine the results
    results = requests.get(url).json()
    
    # Get relevant part of JSON and transform it into a *pandas* dataframe
    # assign relevant part of JSON to venues
    venues = results['response']['venues']

    # tranform venues into a dataframe
    dataframe = json_normalize(venues)
    dataframe.head()
    
    # keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
    if 'location.neighborhood' in filtered_columns:
        filtered_columns.remove('location.neighborhood')
    #print(filtered_columns)
    dataframe_filtered = dataframe.loc[:, filtered_columns]

    # function that extracts the category of the venue
    def get_category_type(row):
        try:
            categories_list = row['categories']
        except:
            categories_list = row['venue.categories']
        
        if len(categories_list) == 0:
            return None
        else:
            return categories_list[0]['name']

    # filter the category for each row
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    # clean column names by keeping only last term
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

    # drops rows with null value
    dataframe_filtered = dataframe_filtered.dropna(subset=["categories"]).reset_index(drop=True)

    # Only pick up restaurants from the venues
    dataframe_filtered = dataframe_filtered[dataframe_filtered['categories'].str.contains('Restaurant')].reset_index(drop=True)

    dataframe_filtered['meanDistance'] = np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - city_coor),1,dataframe_filtered[['lat','lng']].values))

    # print(dataframe_filtered.shape)
    dataframe_filtered.head()
    
    locals()['dataframe_' + str(city).split(',')[0].split(' ')[0]] = dataframe_filtered.copy()

    print(city)
    print("Mean Distance from the City coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - city_coor),1,dataframe_filtered[['lat','lng']].values)))

    

The geograpical coordinate of Toronto, ON, Canada are 43.653963, -79.387207.
Toronto, ON, Canada
Mean Distance from the City coordinates
0.021244259244908338
The geograpical coordinate of Paris, France are 48.8566101, 2.3514992.
Paris, France
Mean Distance from the City coordinates
0.026684869698487795
The geograpical coordinate of New York City, NY are 40.7127281, -74.0060152.
New York City, NY
Mean Distance from the City coordinates
0.009168655594343442


## Explore Venues
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

Since I have id of each restaurant, Let's get the overall rating of each restaurant.

In [54]:
# Combine three dataframe first.
df_union = pd.concat([dataframe_Toronto, dataframe_Paris, dataframe_New], ignore_index=True)
# df_union = df_union.dropna(subset=['city'], inplace=False).reset_index(drop=True)
df_union = df_union[df_union['city'].isin(['Toronto', 'Paris', 'New York'])]
df_union.shape

(46, 16)

In [88]:
venues_list=[]

for index, row in df_union.iterrows():
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(row['id'], CLIENT_ID, CLIENT_SECRET, VERSION)
    #print(url)
    
    # make the GET request
    result = requests.get(url).json()
    
    try:
        print(result['response']['venue']['rating'])
        rate = result['response']['venue']['rating']
    except:
        print('This venue has not been rated yet.')
        rate = None
        
    # return only relevant information for each nearby venue
    
    venues_list.append([row['id'], rate])
    
venues_list

This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not b

[['4adb5472f964a520fc2521e3', None],
 ['4aea2b14f964a520ebb921e3', None],
 ['4dd3d24345dd98b61e74f6af', None],
 ['4ad4c061f964a5209ef720e3', None],
 ['582e3fa85e56b4259dc5f751', None],
 ['4ae31ac7f964a520799021e3', None],
 ['4c62e74a79d1e21e7340d715', None],
 ['4de827e4d4c0faa56428a51a', None],
 ['5005933ee4b0f1ca8f45d868', None],
 ['4b1be39df964a5209cfe23e3', None],
 ['4e9a28752c5b4d6405424a47', None],
 ['4eaaf707775bf08569c7cb49', None],
 ['4fcd193a754a56793a2651a5', None],
 ['4b62563df964a52004432ae3', None],
 ['4c52031694790f478bb8f7a1', None],
 ['53e40006498e6613be82b852', None],
 ['5b808648b9a5a8002c4f6085', None],
 ['4fff3dade4b03c9810038f96', None],
 ['5bd9ebd68ad62e002c189ed7', None],
 ['52d7cd8e498ed5101c1dfbc5', None],
 ['5820d2a11dbf6c6cf432715e', None],
 ['57e293ac498e7cf5068596be', None],
 ['5c6a945a9cadd9002ce586b8', None],
 ['4bbf688f85fbb7135d237267', None],
 ['4c3ee734ca012d7ff21030be', None],
 ['5c9e800d3ba767002c4a4465', None],
 ['4f64efc2e4b0c65cc4c2b877', None],
 

In [55]:
# create a dataframe to store id and rating.
columns = ['id', 'rate']
df_rating = pd.DataFrame(venues_list)
df_rating.columns = columns
# drops rows with null value
df_rating = df_rating.dropna(subset=["rate"]).reset_index(drop=True)
df_rating

,id,rate
0,4adb5472f964a520fc2521e3,7.8
1,4aea2b14f964a520ebb921e3,6.9
2,4ad4c061f964a5209ef720e3,6.0
3,582e3fa85e56b4259dc5f751,6.8
4,4e9a28752c5b4d6405424a47,6.4
5,4b62563df964a52004432ae3,5.5
6,52d7cd8e498ed5101c1dfbc5,7.2
7,57e293ac498e7cf5068596be,7.8
8,4bbf688f85fbb7135d237267,6.8
9,51c82a87498e71cc19da490f,5.6


In [57]:
# Combine two dataframes
df_union = df_union.merge(df_rating, how='left')
# drops rows with null value
# df_union = df_union.dropna(subset=["rate"]).reset_index(drop=True)
df_union.shape

(46, 17)

## Results

There are results I derived from the Foursquare API. It seems that New York City has less restaurant, but close distance and high rate; Paris has far distance; Toronto has more restaurant but low rate. The following here are the pictures of the geoplot generated with folium.

In [86]:
# get the average rationg of this city
df_union['meanRate'] = df_union.groupby(['country', 'city'])['rate'].transform('mean')
df_union.groupby(['city']).agg({'id': 'count', 'meanDistance': 'mean', 'rate': 'mean'}).rename(columns={'rate': 'meanRate', 'id': 'The number of restaurants'}).reset_index()

,city,The number of restaurants,meanDistance,meanRate
0,New York,12,0.009169,7.333333
1,Paris,15,0.026685,6.900000
2,Toronto,19,0.021244,6.566667


#### Let's visualize the Asian restaurants that are nearby

In [79]:
city_name = 'Toronto'
latitude = 43.653963
longitude = -79.387207
city_coor = [latitude, longitude]
#dataframe_Toronto = df_union[df_union['city'] == city_name]

venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

label = folium.Popup("City Coordinate", parse_html=True)
# add City as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup=city_name,
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add markers to map
for lat, lng, label in zip(dataframe_Toronto.lat, dataframe_Toronto.lng, dataframe_Toronto.name):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(venues_map)
    folium.PolyLine([city_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(venues_map)

venues_map

In [75]:
city_name = 'Paris'
latitude = 48.8566101
longitude = 2.3514992
city_coor = [latitude, longitude]
#dataframe_Toronto = df_union[df_union['city'] == city_name]

venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

label = folium.Popup("City Coordinate", parse_html=True)
# add City as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup=city_name,
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add markers to map
for lat, lng, label in zip(dataframe_Paris.lat, dataframe_Paris.lng, dataframe_Paris.name):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(venues_map)
    folium.PolyLine([city_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(venues_map)

venues_map

In [87]:
city_name = 'New York'
latitude = 40.7127281
longitude = -74.0060152
city_coor = [latitude, longitude]
#dataframe_Toronto = df_union[df_union['city'] == city_name]

venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

label = folium.Popup("City Coordinate", parse_html=True)
# add City as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup=city_name,
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add markers to map
for lat, lng, label in zip(dataframe_New.lat, dataframe_New.lng, dataframe_New.name):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(venues_map)
    folium.PolyLine([city_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(venues_map)

venues_map

## Discussion

During the data exploration, Some returned data from the Foursquare API is incomplete. E.g. incorrect city name，missing address and no rating. These kinds of data are excluded from this analysis. The average rate only bases on the restaurants which have a rate.

## Conclusion

Although there are fewest restaurants in New York City, I think Lin will choose New York City as his destination because of close distance and a high rate. Please be aware of this analysis base on the city coordinates as the start point. If he decided his residence, he may update the coordinates to explore again.